# Setup

In [1]:
!pip3 install --upgrade pip -q
!pip3 install nvidia-cublas-cu12==12.4.5.8 nvidia-cuda-cupti-cu12==12.4.127 nvidia-cuda-nvrtc-cu12==12.4.127 nvidia-cuda-runtime-cu12==12.4.127 nvidia-cudnn-cu12==9.1.0.70 nvidia-cufft-cu12==11.2.1.3 nvidia-curand-cu12==10.3.5.147 nvidia-cusolver-cu12==11.6.1.9 nvidia-cusparse-cu12==12.3.1.170 nvidia-nvjitlink-cu12==12.4.127
!pip3 install nltk emoji==0.6.0 -q
!pip3 install torchinfo -q
!pip3 install kagglehub -q
!pip3 install kaggle -q
!pip3 install transformers -q
!pip3 install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 150.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 137.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [2]:
!pip3 install fsspec==2025.3.0

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# list the elements in the current working directory
print("current directory: ", os.getcwd())
#os.chdir('content/drive/MyDrive/DATASCI 266/Project') # Use when working on Berkeley's Google Drive
os.chdir('drive/MyDrive/Knowledge/Berkeley/DATASCI 266/Project') # Use when working on Personal Google Drive
os.listdir()

current directory:  /content


['Project Proposal.gdoc',
 'TweetNormalizer.py',
 '__pycache__',
 'results_baseline',
 'wandb',
 'Tweets_full_normalized.csv',
 'baseline_model',
 'bertweet_model',
 'bertkeep_model',
 'Copy of Lora_(Roberta_Large)_pytorch.ipynb',
 'filtered_tweets.csv',
 'filtered_sample_2500.csv',
 'Normalized_filtered_tweets.csv',
 'Crypto Sentiment Analysis.ipynb',
 'labelled_normalized_dataset.csv',
 'small_train_dataset.csv',
 'small_eval_dataset.csv',
 'small_eval_dataset_filtered.csv',
 'small_train_dataset_filtered.csv',
 'labelled_dataset.csv',
 'bertkeep_model5',
 'bertweet_model_6',
 'bertweet_model_7',
 'bertweet_model_8',
 'bertweet_model_9',
 'bertkeep_model_9',
 'bertkeep_model_10',
 'bertkeep_model_11',
 'Paper',
 'tweets_sentiment.csv',
 'File Index.gsheet',
 'Crypto Sentiment Analysis_Data Preparation.ipynb',
 'Copy of Crypto Sentiment Analysis_BERTweet.ipynb',
 'Copy of Crypto Sentiment Analysis.ipynb',
 'Crypto Sentiment Analysis_BERTweet.ipynb',
 'large_balanced_dataset.csv',
 'Mo

In [5]:
import kagglehub
import pandas as pd
import numpy as np

import torch
from torchinfo import summary # to show the summary of the models
import evaluate # for metric evaluation during training from Huggingface
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from TweetNormalizer import normalizeTweet
from datasets import Dataset # from Huggingface

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType


# Data Loading

In [17]:
try:
  df_large = pd.read_csv('large_balanced_dataset.csv')
except:
  # Load the full dataset if not already in memory
  df2 = pd.read_csv('/content/drive/MyDrive/Project/labelled_dataset.csv')
  df2['tweet'] = df2['tweet'].apply(normalizeTweet)

  # Desired total sample size
  target_size = 100_000

  # Check class distribution
  label_counts = df2['label'].value_counts(normalize=True)
  print("Original label distribution (%):\n", label_counts * 100)

  # Compute number of samples per class
  samples_per_class = (label_counts * target_size).round().astype(int)

  # Sample from each class proportionally
  df_sampled = pd.concat([
      df2[df2['label'] == label].sample(n=n, random_state=42)
      for label, n in samples_per_class.items()
  ])

  # Shuffle the dataset
  df_sampled = df_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

  # Check sampled distribution
  print("\nSampled label distribution (%):\n", df_sampled['label'].value_counts(normalize=True) * 100)

  # Save if desired
  df_sampled.to_csv("large_balanced_dataset.csv", index=False)


In [18]:
# Ensure text is normalized the same way
df_large['tweet'] = df_large['tweet'].apply(normalizeTweet)

# Convert to Hugging Face Dataset
dataset_large = Dataset.from_pandas(df_large, preserve_index=False)

In [19]:
def preprocess_tweet(data):
    encoded = tokenizer.batch_encode_plus(
        data['tweet'],
        max_length=280,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True
    )
    encoded['label'] = data['label']
    return encoded

dataset_large = dataset_large.map(preprocess_tweet, batched=True)
dataset_large.set_format(type='torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

# Model Loading

Here we load the models to evaluate and attach them again to the base model.

In [13]:
# Load model 6
model_dir = "./bertweet_model_6"

# Load the LoRA config first
peft_config = PeftConfig.from_pretrained(model_dir)

# Load base model
baseline_model = AutoModelForSequenceClassification.from_pretrained("./baseline_model")

# Load the fine-tuned LoRA adapter
model = PeftModel.from_pretrained(baseline_model, model_dir)


In [14]:
summary(model)

Layer (type:depth-idx)                                                      Param #
PeftModelForSequenceClassification                                          --
├─LoraModel: 1-1                                                            --
│    └─RobertaForSequenceClassification: 2-1                                --
│    │    └─RobertaModel: 3-1                                               (354,703,360)
│    │    └─ModulesToSaveWrapper: 3-2                                       2,105,350
Total params: 356,808,710
Trainable params: 1,052,675
Non-trainable params: 355,756,035

In [15]:
tokenizer = AutoTokenizer.from_pretrained("./baseline_model")


# LoRA Model Evaluation

In [22]:
from tqdm import tqdm
from sklearn.metrics import classification_report

# Put model in eval mode and move to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

all_preds = []
all_labels = []

# Disable gradients for faster inference
with torch.no_grad():
    for batch in tqdm(dataset_large.iter(batch_size=32), total=len(dataset_large)//32):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        preds = torch.argmax(outputs.logits, axis=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


100%|██████████| 3125/3125 [17:51<00:00,  2.92it/s]


In [28]:
print(classification_report(all_labels, all_preds))

              precision    recall  f1-score   support

           0       0.95      0.59      0.73     62437
           1       0.37      0.83      0.51      9581
           2       0.55      0.77      0.64     27982

    accuracy                           0.67    100000
   macro avg       0.62      0.73      0.63    100000
weighted avg       0.78      0.67      0.68    100000



# Baseline Model Evaluation

In [32]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("./baseline_model")
summary(baseline_model)

Layer (type:depth-idx)                                            Param #
RobertaForSequenceClassification                                  --
├─RobertaModel: 1-1                                               --
│    └─RobertaEmbeddings: 2-1                                     --
│    │    └─Embedding: 3-1                                        51,471,360
│    │    └─Embedding: 3-2                                        526,336
│    │    └─Embedding: 3-3                                        1,024
│    │    └─LayerNorm: 3-4                                        2,048
│    │    └─Dropout: 3-5                                          --
│    └─RobertaEncoder: 2-2                                        --
│    │    └─ModuleList: 3-6                                       302,309,376
├─RobertaClassificationHead: 1-2                                  --
│    └─Linear: 2-3                                                1,049,600
│    └─Dropout: 2-4                                            

In [33]:
# Put model in eval mode and move to appropriate device
baseline_model.to(device)
baseline_model.eval()

all_preds = []
all_labels = []

# Disable gradients for faster inference
with torch.no_grad():
    for batch in tqdm(dataset_large.iter(batch_size=32), total=len(dataset_large)//32):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['label'].to(device)

        outputs = baseline_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        preds = torch.argmax(outputs.logits, axis=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
print(classification_report(all_labels, all_preds))

100%|██████████| 3125/3125 [17:16<00:00,  3.02it/s]

              precision    recall  f1-score   support

           0       0.72      0.56      0.63     62437
           1       0.18      0.40      0.25      9581
           2       0.38      0.42      0.40     27982

    accuracy                           0.51    100000
   macro avg       0.43      0.46      0.43    100000
weighted avg       0.58      0.51      0.53    100000



In [34]:
print(classification_report(all_labels, all_preds))


              precision    recall  f1-score   support

           0       0.72      0.56      0.63     62437
           1       0.18      0.40      0.25      9581
           2       0.38      0.42      0.40     27982

    accuracy                           0.51    100000
   macro avg       0.43      0.46      0.43    100000
weighted avg       0.58      0.51      0.53    100000

